# 4.a - Machine Learning : classification

## Import et préparation du dataset

Les résultats du notebook précédent ont mis en évidence le rôle de l'ensemble des variables du _dataset_ dans le montant du pourboire :  
- `fare_amount`
- `surcharge`
- `tolls_amount`
- `passenger_count`
- `trip_distance`
- `trip_time_in_secs`
- `nuit_jour`
- `jour_semaine`
- `vendor_id`
- `payment_type`
- `vitesse_moyenne`

Nous nous intéresserons ici à prédire **si pourboire il y a**, à partir de ces variables. La prédiciton de son montant le cas échéant fera l'objet d'un autre notebook. À cette fin, nous étudierons les performances de plusieurs modèles de classification, sans fine-tuning, afin de se faire une idée des algorithmes qui performent le mieux sur nos données.

Dans l'ordre, voici les modèles explorés :
1. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[Régression_logistique](#Régression-logistique)</span>
2. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[Arbre_de_décision](#Arbre-de-décision)</span>
3. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[Analyse_linéaire_discriminante](#Analyse-linéaire-discriminante)</span>
4. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[Bayésien naïf](#Bayésien-naïf)</span>
4. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[k-Nearest_Neighbor](#k-Nearest-Neighbor)</span>
5. <span style="position:center; top:10px; right:5px; width:100px; height:90px; margin:10px;">[SVM](#SVM)</span>

Nous laisserons volontairement tous les paramètres des algorithmes par défaut.

In [1]:
%matplotlib inline
import os
#os.chdir('/Users/pierredesmet/Documents/Documents Word/Etudes/UTT/ENSAE/Python pour le Data Scientist')
os.chdir('C:\\Users\\Alexis\\Google Drive\\Documents\\ENSAE\\Semestre 1\\Projet python\\')
import pandas
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,mean_squared_error,r2_score,cohen_kappa_score,classification_report
#import matplotlib.pyplot as plt
import numpy as np
import importation

# Import des données
trips = importation.chargement_donnees()
trips = importation.clean(trips)

# Ajout de 2 variables (cf précédent notebook)
trips['vitesse_moyenne'] = trips['trip_distance']/trips['trip_time_in_secs']
trips['received_tips'] = (trips.tip_amount > 0) 

On sait que :
- L'application de paiement (<span style="color: #fb4141">vendor_id</span>) n'intervient pas dans la formation du pourboire (montré dans le notebook 3), non plus que le <span style="color: #fb4141">medallion</span> ou la <span style="color: #fb4141">hack_license</span> ;
- Les variables `nuit_jour_jour` et <span style="color: #fb4141">nuit_jour_nuit</span> sont linéairement dépendantes, car quand l'une d'elle vaut 1, l'autre vaut nécessairement 0. Il en va de même pour les jours de la semaine (<span style="color: #fb4141">jour_semaine_Monday</span>, ... , `jour_semaine_Sunday`). Idem, la `vitesse_moyenne` d'un taxi est formée à partir des deux variables <span style="color: #fb4141">trip_distance</span> et <span style="color: #fb4141">trip_time_in_secs</span> ;
- le prix de la course (<span style="color: #fb4141">total_amount</span>) inclu le tip éventuellement laissé au chauffeur.
- <span style="color: #fb4141">pickup_datetime</span> et <span style="color: #fb4141">dropoff_datetime</span> contiennent de l'information inutile sur les dates, heures, minutes, et secondes ; on ne va garder que jour/nuit et le jour de la semaine.

À partir de ces constats nous supprimons les variables susnommées en rouge. En outre, les courses payées autrement que par carte bleue n'ont pas permis l'enregistrement d'un éventuel pourboire : on ne gardera donc que les courses payées avec ce moyen :

In [2]:
trips = trips[(trips['payment_type'] == 'CRD')]
trips = pandas.get_dummies(data=trips, columns=['nuit_jour', 'jour_semaine'])
trips = trips.drop(['vendor_id', 'medallion', 'hack_license', 'nuit_jour_jour', 'jour_semaine_Monday',
                    'trip_distance', 'trip_time_in_secs', 'total_amount','pickup_datetime', 'dropoff_datetime',
                    'payment_type','tip_amount'], axis=1)
print('Courses avec pourboire : %.2f%%'%(np.sum(trips.received_tips)/len(trips.received_tips)*100))
trips.head()

Courses avec pourboire : 97.02%


,fare_amount,surcharge,tolls_amount,passenger_count,vitesse_moyenne,received_tips,nuit_jour_nuit,jour_semaine_Friday,jour_semaine_Saturday,jour_semaine_Sunday,jour_semaine_Thursday,jour_semaine_Tuesday,jour_semaine_Wednesday
1,5.0,0.0,0.0,1,0.002375,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,20.5,0.0,0.0,1,0.009792,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12,10.0,0.0,0.0,1,0.004481,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13,7.5,0.0,0.0,6,0.006733,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14,12.5,0.0,0.0,1,0.005000,True,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Validation croisée

Séparons tout d'abord notre jeu de données *trips* en un échantillon d'apprentissage - sur lequel entraîner les modèles, et un échantillon de test - sur lequel faire des prédictions. La validation-croisée (_testset validation_) est utilisable ici car nous ne sommes pas en présence de données ordonnées (séries temporelles). 

In [3]:
train, test = train_test_split(trips, test_size = 0.2)
y_train = train['received_tips']
y_test = test['received_tips']

x_train = train.drop(['received_tips'],axis = 1)
x_test = test.drop(['received_tips'], axis = 1)

## Régression logistique

Nous cherchons à prédire à partir des caractéristiques d'une course x_train(i) si le chauffeur a reçu un pourboire (1) ou non (0). Il s'agit donc d'un problème de classification binaire pour lequel la régression logistique est tout indiquée. On crée donc un modèle de régression logistique et on l'entraînons sur l'échantillon d'entraînement :

In [4]:
# Coefficients
logistic = LogisticRegression()
logistic = logistic.fit(x_train,y_train)
print('Coefficients : ',logistic.coef_,'\nIntercept :',logistic.intercept_,'\n')

def analysis(x,y):
    # Prédictions
    predictions = logistic.predict(x)
    erreurs = np.sum(y != predictions)
    print('\tNombre de prédictions erronées :',erreurs,'/',len(y))
    print('\tPrédictions correctes : %.2f%%'%logistic.score(x,y))
    # Qualité du modèle
    print(classification_report(y, predictions))
    print('Matrice de confusion :\n',confusion_matrix(y, predictions))
    
# Échantillon d'entraînement
print('Échantillon d\'entraînement :')
analysis(x_train,y_train)

# Échantillon test
print('\nÉchantillon test :')
analysis(x_test,y_test)

Coefficients :  [[-0.02595186 -0.02425783  0.02583993 -0.06564043  0.03503838 -0.22364905
  -0.12323402 -0.20117243 -0.21990498  0.03851619  0.00483904  0.07128539]] 
Intercept : [ 4.11298161] 

Échantillon d'entraînement :
	Nombre de prédictions erronées : 11007 / 369876
	Prédictions correctes : 0.97%
             precision    recall  f1-score   support

      False       0.00      0.00      0.00     11007
       True       0.97      1.00      0.98    358869

avg / total       0.94      0.97      0.96    369876



//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Matrice de confusion :
 [[     0  11007]
 [     0 358869]]

Échantillon test :
	Nombre de prédictions erronées : 2755 / 92469
	Prédictions correctes : 0.97%
             precision    recall  f1-score   support

      False       0.00      0.00      0.00      2755
       True       0.97      1.00      0.98     89714

avg / total       0.94      0.97      0.96     92469

Matrice de confusion :
 [[    0  2755]
 [    0 89714]]


___
Que s'est-il passé ? 
Les performances du modèles semblent _a priori_ excellentes, comme en témoignent :
- le taux de prédictions correctes avoisinant les 98% pour les jeux de test et d'apprentissage ;
- la matrice de confusion, dont les coefficients non-diagonaux sont élevés ;
- le `F1_score`, plus proche de 1 que de 0. 
> The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. `F1_score` $= 2 \times \frac{\text{precision } \times \text{ recall}}{\text{precision + recall}}$

Cependant, deux éléments doivent nous alerter : 
- la proportion de pourboire laissés est de 97% : nous sommes en présence "d'unbalanced class". Le score du modèle n'est finalement pas très différent de celui que fournirait un algorithme naïf en prédisant systématiquement la classe modale ;
- la matrice de confusion montre que c'est précisément ce qui s'est passé : le modèle n'a pas appris, et s'est contenté de prédire plus souvent des pourboires, puisque c'est comme ça que 97% des courses se concluent !

Une solution que nous proposons de mettre en place est la constitution d'un jeu contenant autant (ou presque) de courses avec pourboires que de courses sans pourboire.

In [5]:
without_tips = trips.loc[trips.received_tips == False]
with_tips = trips.loc[trips.received_tips == True].sample(len(without_tips))
trips_balanced = pandas.concat([without_tips,with_tips])

train, test = train_test_split(trips_balanced, test_size = 0.2)
y_train = train['received_tips']
y_test = test['received_tips']

x_train = train.drop(['received_tips'],axis = 1)
x_test = test.drop(['received_tips'], axis = 1)

On relance alors la régression logistique sur le nouveau jeu de données équilibré obtenu :

In [6]:
# Coefficients
logistic = logistic.fit(x_train,y_train)

#logistic = logistic.fit(x_train,y_train)
print('Coefficients : ',logistic.coef_,'\nIntercept :',logistic.intercept_,'\n')
    
# Échantillon d'entraînement
print('Échantillon d\'entraînement :')
analysis(x_train,y_train)

# Échantillon test
print('\nÉchantillon test :')
analysis(x_test,y_test)

Coefficients :  [[-0.0237815  -0.03429793  0.01333966 -0.07129146  0.1595065  -0.2903644
  -0.12487918 -0.22423795 -0.24623045 -0.04948964 -0.03860138  0.0419841 ]] 
Intercept : [ 0.67142975] 

Échantillon d'entraînement :
	Nombre de prédictions erronées : 9603 / 22019
	Prédictions correctes : 0.56%
             precision    recall  f1-score   support

      False       0.57      0.50      0.53     11008
       True       0.56      0.63      0.59     11011

avg / total       0.56      0.56      0.56     22019

Matrice de confusion :
 [[5506 5502]
 [4101 6910]]

Échantillon test :
	Nombre de prédictions erronées : 2378 / 5505
	Prédictions correctes : 0.57%
             precision    recall  f1-score   support

      False       0.58      0.51      0.54      2754
       True       0.56      0.63      0.59      2751

avg / total       0.57      0.57      0.57      5505

Matrice de confusion :
 [[1402 1352]
 [1026 1725]]


Cette fois le modèle semble avoir appris (pas beaucoup). Le taux de prédictions correctes sur l'échantillon test avoisine les 57%, ce qui n'est tout de même pas très bon.

À partir de là, il est possible de faire de la prédiction pour une nouvelle course.
Prenons par exemple la course fictive suivante :

| `fare_amount` | `surcharge` | `tolls_amount` | `passenger_count` | `vitesse_moyenne` | `trip_jour_nuit` | `jour_semaine_Saturday` |
|:-----------:|:---------:|:------------:|:---------------:|:-------------:|:-----------------:|:--------------:|:-------------:|:--------------------:|:---------------:|
|      3     |     0     |       0      |        4        |      0.00175      |       0        |        1       |  

Le chauffeur d'une telle course peut-il s'attendre à un pourboire ?

In [7]:
data = {'fare_amount':3, 'surcharge':0, 'tolls_amount':0, 'passenger_count':4,
        'vitesse_moyenne': 0.00175,'nuit_jour_nuit':0,
        'jour_semaine_Tuesday':0,'jour_semaine_Wednesday':0, 'jour_semaine_Thursday':0,
        'jour_semaine_Friday':0,'jour_semaine_Saturday':1, 'jour_semaine_Sunday':0}
my_single_course = pandas.DataFrame(data = data, index = [0])
print('Le chauffeur peut s\'attendre à un pourboire :',logistic.predict(my_single_course))

Le chauffeur peut s'attendre à un pourboire : [False]


## Arbre de décision

Dans la perspective de comparer le modèle précédent à un autre, nous essayons à présent de prédire si le chauffeur recevra un pourboire _via_ un arbre de décision.

In [8]:
my_tree = DecisionTreeClassifier()
my_tree = my_tree.fit(x_train,y_train)

# Prédiction de la course précédente :
print('Le chauffeur peut s\'attendre à un pourboire :',my_tree.predict(my_single_course))

def analysis_tree(x,y):
    # Prédictions
    predictions = my_tree.predict(x)
    erreurs = np.sum(y != predictions)
    print('\tNombre de prédictions erronées :',erreurs,'/',len(y))
    # Qualité du modèle
    print(classification_report(y_test, predictions))
    print('Matrice de confusion :\n',confusion_matrix(y,predictions))

# Prédictions sur l'ensemble test :
analysis_tree(x_test, y_test)

Le chauffeur peut s'attendre à un pourboire : [False]
	Nombre de prédictions erronées : 2666 / 5505
             precision    recall  f1-score   support

      False       0.52      0.52      0.52      2754
       True       0.52      0.51      0.51      2751

avg / total       0.52      0.52      0.52      5505

Matrice de confusion :
 [[1429 1325]
 [1341 1410]]


Les résultats d'un arbre étant excessivement mauvais, on s'abstiendra d'étudier le modèle de random forest, celui-ci étant construit sur des arbres de décision.

## Analyse linéaire discriminante

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis(solver = 'lsqr')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
erreurs = np.sum(y_test != predictions)
print('Nombre de prédictions erronées :',erreurs,'/',len(y_test))
print(classification_report(y_test, predictions))
print('Matrice de confusion :\n',confusion_matrix(y_test,predictions))

Nombre de prédictions erronées : 2377 / 5505
             precision    recall  f1-score   support

      False       0.58      0.51      0.54      2754
       True       0.56      0.63      0.59      2751

avg / total       0.57      0.57      0.57      5505

Matrice de confusion :
 [[1397 1357]
 [1020 1731]]


## Bayésien naïf

In [10]:
from sklearn.naive_bayes import GaussianNB
naif = GaussianNB()
naif.fit(x_train, y_train)
predictions = naif.predict(x_test)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

             precision    recall  f1-score   support

      False       0.53      0.75      0.62      2754
       True       0.57      0.34      0.43      2751

avg / total       0.55      0.54      0.52      5505

[[2056  698]
 [1817  934]]


## k-Nearest Neighbor

In [11]:
neigh = KNeighborsClassifier()
neigh.fit(x_train, y_train)
predictions = neigh.predict(x_test)
print(neigh)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

      False       0.52      0.52      0.52      2754
       True       0.52      0.52      0.52      2751

avg / total       0.52      0.52      0.52      5505

[[1420 1334]
 [1309 1442]]


## SVM

In [12]:
model = SVC()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
print(model)
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

      False       0.57      0.48      0.52      2754
       True       0.55      0.63      0.59      2751

avg / total       0.56      0.56      0.55      5505

[[1320 1434]
 [1011 1740]]


In [13]:
synthese_algo = {
    'Logistic_model': {'score_train': logistic.score(x_train,y_train), 'score_test': logistic.score(x_test,y_test)},
    'Decision_tree': {'score_train': my_tree.score(x_train, y_train), 'score_test': my_tree.score(x_test, y_test)},
    'Discriminant_analysis': {'score_train': clf.score(x_train, y_train), 'score_test': clf.score(x_test, y_test)},
    'Naive_Bayes_classifier': {'score_train': naif.score(x_train, y_train), 'score_test': naif.score(x_test, y_test)},
    'k-Nearest_Neighbor': {'score_train': neigh.score(x_train, y_train), 'score_test': neigh.score(x_test, y_test)},
    'SVM': {'score_train': model.score(x_train, y_train), 'score_test': model.score(x_test, y_test)}
}
pandas.DataFrame(synthese_algo)

,Decision_tree,Discriminant_analysis,Logistic_model,Naive_Bayes_classifier,SVM,k-Nearest_Neighbor
score_test,0.515713,0.568211,0.568029,0.543143,0.555858,0.519891
score_train,0.995004,0.564013,0.563877,0.543621,0.581180,0.698124


On peut conclure que prédire qu'un trajet ne conduira pas à un pourboire est un problème difficile. On peut supposer que c'est parce qu'il n'existe pas de structure qui explique cette action, et qu'à partir des données que l'on possède, ces trajets apparaissent comme totalement aléatoire.

En revanche, nous allons voir dans le prochain notebook que prédire la valeur du pourboire est un problème plus simple.

On note qu'on pourrait à priori penser qu'il est plus simple de prédire pourboire/pas de pourboire que de donner la valeur de celui-ci. On verra que ce n'est pas le cas ici.